# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The CNN-RNN architecture that is used to solve the image captioning problem is implemented by following a research paper as recommended by the instructor (https://arxiv.org/pdf/1411.4555.pdf)[1]. The architecture is described in Figure 3 of the paper. Following the figure the most important part of the implementation has to make two features combine in the LSTM network i-e, a feature tensor originating out of the CNN model has to be concatenated with the caption feature tensor. The CNN model or the model where image features are extracted is referred to as an encoder by the authors of the paper. And RNN (LSTM) part of the architecture is referred to as the decoder. The decoder part has the responsibility to merge input features (captions) and image features coming out of the encoder network.

According to the authors: CNN can be any pre-trained model that is trained on ImageNet or a large state-of-the-art network trained on a large number of classes that are common with the COCO dataset as well. However, the instructors of the course chose to go with Resnet50 with the last layer replaced with a linear untrained layer that would be trained during training sessions (transfer learning). In summary, the training of the image captioning network involves fixed parameters for Resnet50 with the last linear to match the feature embedding dimension of the LSTM inputs. The parameters of the linear part of the encoder and the entire word predicting network based on LSTM are part of the entire training network architecture.

The authors of the paper [1] didn't describe training hyperparameters in detail however, an important research paper was consulted (https://www.researchgate.net/publication/342352783_Hyperparameter_Analysis_for_Image_Captioning)[2] that describes which training parameters matters the most during training of the CNN-RNN (CNN-LSTM) architecture. I have used 256 to embedding dimensions for words to match the image feature tensor. Changing embedding size/dim would require experimentation with appropriate image transforms as well. Furthermore, the hidden size was kept to 256 as well. As described by the authors of [2] between hidden size embedding dimensions namely 256, 512, and 1024 there were no significant differences observed in the performance of LSTM network combination with Resnet 18, 50, 101. But to keep sufficient performance and training time trade-off intact I choose 256 over any other hidden dimension size.

The authors of paper(https://arxiv.org/pdf/1502.03044.pdf)[3] used a batch size of 64, I have trained CNN-RNN with batch sizes of 128 and 32 both were showing similar results after 1 epoch. So, I skipped to 128 batch size because it requires a complete day to train and for the purpose of the task a batch size of 32 provides right kind of trade-off between time to train, local machine and provided cloud machine computational requirements and sufficient accuracy. On local machine with Quadro P2000 I was able to train the network in 4 hours approximately with a batch size of 32 and 3-Epochs. On provided cloud environment with K80 Tesla the network took almost 12 hours to train. 

It was observed that network trained after 3rd epoch resulted in better approximation and more accurate results in terms of sentences explaining the pictures, but caching early lowest-scored loss function model in epoch 1 performed well on good sentence structure albeit quite less accurate as output by the network. As an example during Epoch-1 a model with a loss of less than 2.2 was saved and it performed somewhat similar in sentence structure as the fully trained model. But with more mistakes on the test set in describing the picture. I didn't explore validation loss individually but it would be worth the investigation if early stopping models are similar performant then the fully trained network. The last important parameter is vocabulary size which in my opinion default/provided size of 5 as described by the authors of the paper[1] is sufficient to provide enough vocabulary to the network while keeping the dataset (vocabulary size) manageable.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** As original Resnet(https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf) was trained on 224x224 therefore parameters mentioned by the instructor in image caption project notebooks including Normalize and Random crop, flip are recommended way to get most appropriate results out of pre-trained network. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** As described in the answer to question 1, trainable parameters includes the last layer of encoder-CNN and all trainable parameters on the decoder-RNN side (weights and biases of LSTM and Linear fully-connected layer).

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** According to the paper[1] fixed learning and no-momentum SGD optimizer was used by the author to train the CNN-RNN model. However, author of paper[2] and [3] also suggested by the instructor used Adam and it was also selected because it has momentum properties to find best global minimum.

In [2]:
from pathlib import Path
current_file_path = Path.cwd()
model_encoder_file = 'models/img_cap_2_2_encoder_b32.pt'
model_decoder_file = 'models/img_cap_2_2_decoder_b32.pt'
model_encoder_file_20 = 'models/img_cap_2_3_encoder_b32.pt'
model_decoder_file_20 = 'models/img_cap_2_3_decoder_b32.pt'
models_dir = 'models'
tmp = current_file_path / model_encoder_file
model_save_path = current_file_path / models_dir
print(model_save_path, tmp)

/home/ali/PycharmProjects/CVND/CVND---Image-Captioning-Project/models /home/ali/PycharmProjects/CVND/CVND---Image-Captioning-Project/models/img_cap_2_2_encoder_b32.pt


In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
# sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 256         # number of features in hidden state of the RNN decoder
num_epochs = 3            # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params_to_update = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params=params_to_update, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:31<00:00, 13215.82it/s]
/home/ali/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ali/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ali/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
print(f"DecoderRNN: {encoder}, EncoderCNN: {decoder}")


In [ ]:
print(list(decoder.named_parameters()))

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

# old_time = time.time()
# response = requests.request("GET", 
#                             "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                             headers={"Metadata-Flavor":"Google"})

saved_once_2 = True
saved_once_1 = True
for epoch in range(1, num_epochs+1):
    # hc = decoder.init_hidden(batch_size)
    for i_step in range(1, total_step+1):
        
        # if time.time() - old_time > 60:
        #     old_time = time.time()
        #     requests.request("POST", 
        #                      "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                      headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
  
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        # (h, c) = hc
        # outputs, hc = decoder(features, captions, (h.detach(), c.detach()))
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        # loss.backward(retain_graph=True)
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

        if loss.item() <= 2.2 and (saved_once_2 is True):
            saved_once_2 = False
            tmp_model_encoder = current_file_path / model_encoder_file
            tmp_model_decoder = current_file_path / model_decoder_file
            torch.save(encoder.state_dict(), tmp_model_encoder)
            torch.save(decoder.state_dict(), tmp_model_decoder)
            print(f"file save: {tmp_model_encoder}")
            print(f"file save: {tmp_model_decoder}")

        if loss.item() <= 2.0 and (saved_once_1 is True):
            saved_once_1 = False    
            tmp_model_encoder = current_file_path / model_encoder_file_20
            tmp_model_decoder = current_file_path / model_decoder_file_20
            torch.save(encoder.state_dict(), tmp_model_encoder)
            torch.save(decoder.state_dict(), tmp_model_decoder)
            print(f"file save: {tmp_model_encoder}")
            print(f"file save: {tmp_model_decoder}")    

    # Save the weights.
    if epoch % save_every == 0:
        decoder_path_str = 'decoder_{}.pt'.format(epoch)
        decoder_save_path = model_save_path / decoder_path_str
        encoder_path_str = 'encode_{}.pt'.format(epoch)
        encoder_save_path = model_save_path / encoder_path_str
        torch.save(decoder.state_dict(), decoder_save_path)
        torch.save(encoder.state_dict(), encoder_save_path)
        # torch.save(decoder.state_dict(), os.path.join('models', 'decoder_%d.pkl' % epoch))
        # torch.save(encoder.state_dict(), os.path.join('models', 'encode_%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/12942], Loss: 4.7777, Perplexity: 118.8320
Epoch [1/3], Step [200/12942], Loss: 4.3688, Perplexity: 78.95027
Epoch [1/3], Step [300/12942], Loss: 4.0660, Perplexity: 58.32557
Epoch [1/3], Step [400/12942], Loss: 3.7838, Perplexity: 43.98076
Epoch [1/3], Step [500/12942], Loss: 3.6938, Perplexity: 40.1956
Epoch [1/3], Step [600/12942], Loss: 3.6986, Perplexity: 40.39118
Epoch [1/3], Step [700/12942], Loss: 3.9924, Perplexity: 54.18491
Epoch [1/3], Step [800/12942], Loss: 3.8507, Perplexity: 47.02384
Epoch [1/3], Step [900/12942], Loss: 3.6757, Perplexity: 39.4778
Epoch [1/3], Step [1000/12942], Loss: 3.7281, Perplexity: 41.5988
Epoch [1/3], Step [1100/12942], Loss: 3.3741, Perplexity: 29.1985
Epoch [1/3], Step [1200/12942], Loss: 3.4698, Perplexity: 32.1297
Epoch [1/3], Step [1300/12942], Loss: 3.4198, Perplexity: 30.5619
Epoch [1/3], Step [1400/12942], Loss: 3.7368, Perplexity: 41.9642
Epoch [1/3], Step [1500/12942], Loss: 3.1741, Perplexity: 23.9049
Epoch [1/3],

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.